In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\NLP\\NLP Projects\\Churn-Prediction'

In [14]:
import pandas as pd

In [15]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    root_dir: Path
    local_data_file: Path
    train_csv: Path
    test_csv: Path

In [16]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(self,config_path=CONFIG_FILE_PATH,schema_path=SCHEMA_FILE_PATH,param_path=PARAMS_FILE_PATH):
        self.config_path=read_yaml(config_path)
        self.schema_path=read_yaml(schema_path)
        self.param_path=read_yaml(param_path)
        
        create_directories([self.config_path.artifacts_root])
        
    
    def get_data_datatransformation(self) -> DataTransformationConfig:
        config=self.config_path.data_transformation
        create_directories([config.root_dir])
        data_tranformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            train_csv=config.train_csv,
            test_csv=config.test_csv
        )
        return data_tranformation_config
        
        
        
        

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
from mlProject.Logging import get_logger
logger=get_logger(__name__)

In [25]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config
        
        pass
    
    def drop_columns(self):
        df=pd.read_csv(self.config.local_data_file)
        df.drop(columns=['City','Signup_Quarter'],inplace=True)
        return df
    
    def train_test_split(self):
        df=self.drop_columns()
        train,test=train_test_split(df,test_size=0.2,random_state=42)
        train.to_csv(self.config.train_csv,index=False)
        test.to_csv(self.config.test_csv,index=False)
        
        logger.info("data splitted in to training and test sets")
        
        
        
        

In [26]:
obj_manager=ConfigurationManager()
data_tranformation_config=  obj_manager.get_data_datatransformation()

obj_trans=DataTransformation(data_tranformation_config)

obj_trans.train_test_split()

2026-01-12 12:46:59,583 | INFO | mlProject.utils.common | common.py:31 | yaml file: config\config.yaml loaded successfully
2026-01-12 12:46:59,585 | INFO | mlProject.utils.common | common.py:31 | yaml file: schema.yaml loaded successfully
2026-01-12 12:46:59,591 | INFO | mlProject.utils.common | common.py:31 | yaml file: params.yaml loaded successfully
2026-01-12 12:46:59,592 | INFO | mlProject.utils.common | common.py:51 | created directory at: artifacts
2026-01-12 12:46:59,594 | INFO | mlProject.utils.common | common.py:51 | created directory at: artifacts/data_transformation
2026-01-12 12:47:00,128 | INFO | __main__ | 1705154133.py:18 | data splitted in to training and test sets
